## Setup and Import Libraries
### Required libraries for geospatial data processing and visualization.

In [5]:
import os
import cv2
import json
import rasterio
import warnings
import numpy as np
import geopandas as gpd
from pathlib import Path
from osgeo import gdal
from math import radians, cos, sqrt
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline
from shapely.geometry import box, Polygon
from PIL import Image, ImageDraw
from PIL.TiffTags import TAGS
from rasterio.errors import NotGeoreferencedWarning
warnings.filterwarnings('ignore', category=NotGeoreferencedWarning)


## Dataset Analysis Function
### Function to analyze the contents of a dataset directory, counting TIF images and summarizing file types.

`Discription:`
This code scans a directory and analyzes its contents by:
1. Finding all files in 'dataset/' directory
2. Counting how many files have each extension type (like .tif, etc.)
3. Specifically reports the number of .tif image files found
4. Shows a breakdown listing each file extension and its count

The code uses Path() for directory scanning and Counter() for tallying file types, organizing the results into a simple frequency count report.

In [ ]:
def analyze_dataset(directory_path):
    files = list(Path(directory_path).glob('*'))
    extensions = Counter(f.suffix for f in files)
    
    print(f"Total TIF images: {extensions.get('.tif', 0)}")
    print("\nFile types in dataset:")
    for ext, count in extensions.items():
        print(f"{ext}: {count} files")

analyze_dataset('dataset/.')

## Image Analysis Function
### Displays key metrics and visualization of a TIF file, including dimensions, data type, and pixel values.

`Description:`
This code analyzes and displays TIF (Tagged Image Format) images with these key operations:

1. Image Reading:
- Opens TIF file using rasterio
- Gets dimensions (width, height) and number of bands
- For RGB images (3+ bands): reads bands 1,2,3 and transposes to correct shape (height,width,channels)
- For grayscale: reads only band 1

2. Display Logic:
- For RGB: normalizes by dividing by max value (img/img.max())
- For grayscale: uses 'gray' colormap
- Displays using matplotlib with a 10x10 figure size
- Removes axis for cleaner visualization

The key conditional logic is:
```python
img = src.read([1,2,3]).transpose(1,2,0) if bands >= 3 else src.read(1)
plt.imshow(img/img.max() if bands >= 3 else img, cmap='gray' if bands==1 else None)
```
This handles both RGB and grayscale images appropriately.

In [3]:
def analyze_tif(file_path):
    """Display TIF image and basic info"""
    with rasterio.open(file_path) as src:
        width = src.width
        height = src.height
        bands = src.count
        
        print(f"Size: {width}x{height}")
        print(f"Bands: {bands}")

        # Read image based on number of bands
        img = src.read([1,2,3]).transpose(1,2,0) if bands >= 3 else src.read(1)
        
        plt.figure(figsize=(10,10))
        plt.imshow(img/img.max() if bands >= 3 else img, cmap='gray' if bands==1 else None)
        plt.title('TIF Image')
        plt.axis('off')
        plt.show()

### Analyzing multiple TIF images from the dataset to compare dimensions and pixel characteristics.

In [ ]:
analyze_tif('data/20241014aC0822230w275445n.tif')
analyze_tif('dataset/20241014aC0822145w275530n.tif')
analyze_tif('dataset/20241014aC0822145w274500n.tif')

---

## Coordinate Analysis and Center Calculation
### Analyzes and compares coordinates between TIF image and shapefile, calculating center points.

`Description:`
This code extracts geographic coordinates from a GeoTIFF file:

1. `get_center`: Calculates midpoint between bounds
- Takes coordinates of bounding box (left, bottom, right, top)
- Returns center point (latitude, longitude)

2. `get_coords`: Extracts coordinates from GeoTIFF
- Opens file with rasterio
- Gets bounds and CRS (Coordinate Reference System)
- Calculates and prints center coordinates

The bounds define the geographic extent of the image, and the center point helps locate the image's central position on Earth.

In [ ]:
def get_center(left, bottom, right, top):
   """Calculate center Points"""
   return (top + bottom)/2, (left + right)/2

def get_coords(tif_path):
   """Get coordinates from TIF file"""
   with rasterio.open(tif_path) as src:
       bounds = src.bounds
       print(f"Bounds: {bounds}")
       print(f"CRS: {src.crs}")
       
       center_lat, center_lon = get_center(bounds.left, bounds.bottom, 
                                         bounds.right, bounds.top)
       print('-------------------------------')
       print(f"According to Image.TIF Center Latitude, Longitude: ({center_lat}, {center_lon})")
       print('-------------------------------')

get_coords('data/20241014aC0822230w275445n.tif')

# Image Tiling and Grid Visualization

## Grid Visualization (640*640)
`Description:`
This code creates a grid overlay on an image:

1. Opens image and creates drawing object
2. Iterates through image in steps of `size` (640 pixels)
3. Draws black rectangles with 5px width at each grid position
4. Returns modified image with grid overlay

This helps divide large images into smaller, equal-sized sections for analysis or processing.

In [ ]:
def draw_grid(image_path, size=640):
    """Draw grid on image"""
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    for x in range(0, img.width, size):
        for y in range(0, img.height, size):
            draw.rectangle([(x,y), (x+size, y+size)], 
                         outline='black', width=5)
    return img

# Display image 
plt.figure(figsize=(10,10))
plt.imshow(draw_grid("data/20241014aC0822230w275445n.tif"))
plt.axis('off')
plt.show()

## Tile Generation
`Description:`
This code splits a large image into smaller tiles while preserving geographic metadata:

1. Reads source image and shapefile
2. Calculates pixel size relative to geographic coordinates
3. Iterates through image in tile_size steps
4. For each tile:
   - Crops image section
   - Calculates geographic bounds
   - Saves tile and metadata
5. Creates GeoDataFrame with tile info and saves to shapefile

Key calculations:
- Pixel size = geographic extent / image dimensions
- Tile bounds = base coordinates + (pixel index × pixel size)

In [7]:
def create_tiles(shp_path, img_path, output_dir, tile_size=640):
   """Split image into tiles and save metadata"""
   # Read files
   gdf = gpd.read_file(shp_path)
   img = Image.open(img_path)
   os.makedirs(output_dir, exist_ok=True)

   # Calculate dimensions
   extent = gdf.total_bounds
   width, height = img.size
   px_size_x = (extent[2] - extent[0]) / width 
   px_size_y = (extent[3] - extent[1]) / height

   tiles = []
   count = 1

   # Create tiles
   for i in range(0, width, tile_size):
       for j in range(0, height, tile_size):
           # Crop and save tile
           tile = img.crop((i, j, i + tile_size, j + tile_size))
           tile_path = os.path.join(output_dir, f"tile_{count}.tif")
           tile.save(tile_path)

           # Calculate bounds
           minx = extent[0] + i * px_size_x
           maxx = extent[0] + (i + tile_size) * px_size_x
           maxy = extent[3] - j * px_size_y
           miny = extent[3] - (j + tile_size) * px_size_y

           tiles.append({
               "tile_name": tile_path,
               "geometry": box(minx, miny, maxx, maxy)
           })
           count += 1

   # Save metadata
   tile_gdf = gpd.GeoDataFrame(tiles, crs=gdf.crs)
   tile_gdf.to_file(os.path.join(output_dir, "tile_metadata.shp"))

create_tiles("data/tile_index_20241014a_RGB.shp",
           "data/20241014aC0822230w275445n.tif", 
           "tiles_with_metadata")

## Analyze Generated Tiles
### Checking the contents and counts of the tiled dataset.

In [ ]:
analyze_dataset('tiles_with_metadata/.')

## Tile Analysis
### Analyzing sample tiles from different locations in the original image.

In [ ]:
analyze_tif('tiles_with_metadata/tile_18.tif')
print("-----------------------------------------------------------------------------------------------")
analyze_tif('tiles_with_metadata/tile_64.tif')

## Convert TIF Tiles to JPEG
### Converting all generated TIF tiles to JPEG format for easier viewing.

`Description:`
This code converts TIF files to JPG format:

1. Creates output directory
2. For each TIF file:
   - Opens file
   - Converts to RGB colorspace
   - Saves as JPG 
   - Uses case-insensitive match for .tif extension
3. Returns completion message

Required imports: pathlib.Path, PIL.Image

Purpose: Batch converts TIF images to more commonly used JPG format while preserving color information.

In [ ]:
# def convert_tif_to_jpg(input_dir: str, output_dir: str):
#    Path(output_dir).mkdir(exist_ok=True)
   
#    [Image.open(tif).convert('RGB').save(Path(output_dir) / f"{tif.stem}.jpg", 'JPEG') 
#     for tif in Path(input_dir).glob('*.[tT][iI][fF]')]
   
#    return f"Converted: Available in {output_dir}"

# print(convert_tif_to_jpg("tiles_with_metadata", "jpg_output_dataset"))

## Coordinate Analysis and Center Calculation

### Tile Match Distance Calculator with Center-Based Visualization

# Formulas Used in the Code

### 1. Pixel-to-Coordinate Transformation (`pixel_to_coord` function)
This function converts pixel indices `(x, y)` into geographical coordinates `(lon, lat)` using a geotransformation matrix.

#### Formula:
- **Longitude**:  
  Start with `GT_0`, then add:
  - `(x + 0.5) * GT_1` (pixel index `x` adjusted and multiplied by pixel width in longitude),
  - `(y + 0.5) * GT_2` (pixel index `y` adjusted and multiplied by rotation term).

  ```bash
  lon = GT_0 + (x + 0.5) * GT_1 + (y + 0.5) * GT_2
  ```

- **Latitude**:  
Start with `GT_3`, then add:
- `(x + 0.5) * GT_4` (pixel index `x` adjusted and multiplied by rotation term),
- `(y + 0.5) * GT_5` (pixel index `y` adjusted and multiplied by pixel height in latitude).

```bash
lat = GT_3 + (x + 0.5) * GT_4 + (y + 0.5) * GT_5
```



### 2. Coordinate-to-Pixel Transformation (`coord_to_pixel` function)
This function calculates pixel indices `(x, y)` from geographical coordinates `(lon, lat)`.

#### Formula:
- **Pixel \(x\)-coordinate**:  
Subtract `GT_0` from `lon`, then divide by `GT_1`.  
Take the integer part (floor):  
```bash
x = floor((lon - GT_0) / GT_1)
```


- **Pixel \(y\)-coordinate**:  
Subtract `GT_3` from `lat`, then divide by `GT_5`.  
Take the integer part (floor):  

```bash
y = floor((lat - GT_3) / GT_5)
```



### 3. Euclidean Distance Calculation
This formula calculates the distance between two points in pixels.

#### Formula:
1. Subtract the center coordinates:
 - `dx = match_center_x - center_x`
 - `dy = match_center_y - center_y`
2. Calculate the distance using the Pythagorean theorem:  

```bash
distance = sqrt(dx^2 + dy^2)
```

---

## Code Description

### 1. Functions
- **`pixel_to_coord(x, y, dataset)`**:  
  Converts pixel indices `(x, y)` to geographical coordinates `(lon, lat)` using the affine geotransformation matrix from the dataset.

- **`coord_to_pixel(lon, lat, dataset)`**:  
  Converts geographical coordinates `(lon, lat)` back to pixel indices `(x, y)` for referencing pixels in the image.



### 2. Template Matching
The `visualize_tile_matches` function performs template matching using OpenCV’s `matchTemplate`. It identifies regions in the large image that match the provided tile, based on a confidence threshold.



### 3. Visualization
- Draws rectangles around matched regions in the large image.
- Annotates the geographical coordinates of matched regions on the image.
- Computes and displays the Euclidean distance from the center of the large image to the center of each match.



### 4. Sorting Matches
Matches are sorted by their distance from the center of the large image, providing a prioritized list of matches.



### 5. Output
- A visual output of the large image with matches highlighted and annotated.
- A sorted list of matches, including:
  - Pixel locations
  - Euclidean distances
  - Confidence scores
  - Geographic coordinates (longitude and latitude).



## Summary
This code integrates geospatial concepts with computer vision techniques, allowing for the mapping of pixel-based image analysis to geographic coordinates.

In [ ]:
import math
def pixel_to_coord(x, y, dataset):
    geotransform = dataset.GetGeoTransform()
    # Adjust coordinates to the center of the pixel
    lon = geotransform[0] + ((x + 0.5) * geotransform[1]) + ((y + 0.5) * geotransform[2])
    lat = geotransform[3] + ((x + 0.5) * geotransform[4]) + ((y + 0.5) * geotransform[5])
    return lon, lat

def coord_to_pixel(lon, lat, dataset):
    geotransform = dataset.GetGeoTransform()
    x = int((lon - geotransform[0]) / geotransform[1])
    y = int((lat - geotransform[3]) / geotransform[5])
    return x, y

def visualize_tile_matches(large_image_path, tile_image_path, threshold=0.8):
    # Read images with error checking
    large_img = cv2.imread(large_image_path)
    if large_img is None:
        raise ValueError(f"Failed to load {large_image_path}")
        
    tile_img = cv2.imread(tile_image_path)
    if tile_img is None:
        raise ValueError(f"Failed to load {tile_image_path}")

    large_gray = cv2.cvtColor(large_img, cv2.COLOR_BGR2GRAY)
    tile_gray = cv2.cvtColor(tile_img, cv2.COLOR_BGR2GRAY)
    
    # Template matching
    result = cv2.matchTemplate(large_gray, tile_gray, cv2.TM_CCOEFF_NORMED)
    locations = np.where(result >= threshold)
    
    # Image dimensions
    h, w = tile_gray.shape
    height, width = large_gray.shape
    center_x, center_y = width // 2, height // 2

    
    dataset = gdal.Open('data/20241014aC0822230w275445n.tif') # Change iamge path to your image 'data/20241014aC0822230w275445n.tif'
    # geotransform = dataset.GetGeoTransform()
    # pixel_width = abs(geotransform[1])
    # pixel_height = abs(geotransform[5])
    
    output = large_img.copy()
    cv2.circle(output, (center_x, center_y), 10, (0, 0, 255), -1)
    
    matches = []
    for pt in zip(*locations[::-1]):
        x, y = pt
        confidence = result[y, x]
        
        match_center_x = x + w // 2
        match_center_y = y + h // 2
        lon, lat = pixel_to_coord(match_center_x, match_center_y, dataset)
        
        cv2.rectangle(output, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        dx = match_center_x - center_x
        dy = match_center_y - center_y
        distance = math.sqrt(dx**2 + dy**2)
        
        cv2.line(output, (center_x, center_y), (match_center_x, match_center_y), (255, 0, 0), 2)
        
        text = f"Lat: {lat:.6f}, Lon: {lon:.6f}"
        cv2.putText(output, text, (x, y - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (36, 255, 12), 2)
        
        matches.append({
            'location': (x, y),
            'distance': distance,
            'confidence': confidence,
            'coordinates': (lon, lat)
        })
    
    matches.sort(key=lambda m: m['distance'])
    
    plt.figure(figsize=(15, 15))
    plt.imshow(cv2.cvtColor(output, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Tile Matches with Distances and Coordinates')
    plt.show()
    
    return matches
# Change iamge path to your image 'data/20241014aC0822230w275445n.tif'
matches = visualize_tile_matches('data/20241014aC0822230w275445n.tif', 'tiles_with_metadata/tile_12.tif')
print("\nTop matches with coordinates:")
for match in matches[:3]:
    print(f"Location: {match['location']}")
    print(f"Distance: {match['distance']:.1f}px")
    print(f"Coordinates: {match['coordinates']}\n")


## Labeling 
#### `Description:`
A $mask$ $image$ is a grayscale or binary image used to selectively hide or show parts of another image. Black pixels (value 0) typically indicate hidden/masked areas, while white pixels (value 1) show visible areas. Masks are commonly used in image processing for:

- Segmentation
- Background removal
- Blending multiple images
- Applying effects to specific regions

The example shown is a binary mask with all black pixels, indicating complete masking of the underlying content.  
This code processes and visualizes semantic segmentation masks:

1. Loads RGB image and mask image
2. Normalizes mask values to 0-1 range
3. Creates colored mask overlay (red channel)
4. Blends original and mask images with weights 0.7 and 0.3
5. Displays three images side by side:
   - Original RGB
   - Grayscale mask
   - Overlay (RGB + mask)

Key operations:
```python
# Normalize mask
mask_image = mask_image / mask_image.max()

# Create colored overlay
color_mask = np.zeros_like(rgb_image)
color_mask[..., 0] = mask_image * 255

# Blend images
overlay_image = cv2.addWeighted(rgb_image, 0.7, color_mask, 0.3, 0)
```

This visualization helps validate semantic segmentation results by showing labeled regions overlaid on the original image.

In [14]:
# import cv2
# import matplotlib.pyplot as plt
# import numpy as np

# rgb_image = cv2.imread('road lableing.png-mask-semantic/train/tile_640_0_jpg.rf.dbade169a66a9ffd26d4b443aed79d4e.jpg')
# rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_BGR2RGB) 

# mask_image = cv2.imread('road lableing.png-mask-semantic/train/tile_640_0_jpg.rf.dbade169a66a9ffd26d4b443aed79d4e_mask.png', cv2.IMREAD_GRAYSCALE)
# mask_image = mask_image / mask_image.max() 
# color_mask = np.zeros_like(rgb_image)
# color_mask[..., 0] = mask_image * 255 

# overlay_image = cv2.addWeighted(rgb_image, 0.7, color_mask, 0.3, 0)

# plt.figure(figsize=(15, 10))

# plt.subplot(1, 3, 1)
# plt.title('RGB Image')
# plt.imshow(rgb_image)
# plt.axis('off')

# plt.subplot(1, 3, 2)
# plt.title('Mask Image')
# plt.imshow(mask_image, cmap='gray')
# plt.axis('off')

# plt.subplot(1, 3, 3)
# plt.title('Overlay Image')
# plt.imshow(overlay_image)
# plt.axis('off')

# plt.tight_layout()
# plt.show()


---
---
---
---
---

# LRS (Linear Refrence System)

### **1. GSD (Ground Sample Distance) Calculation**
This formula calculates the ground sample distance (meters per pixel) for the X and Y directions.

#### Formula:
- `gsd_x = pixel_scale_x * meters_per_degree_lon`
- `gsd_y = pixel_scale_y * meters_per_degree_lat`

#### Variables:
- `pixel_scale_x` and `pixel_scale_y`: Pixel scale in degrees per pixel (from metadata).
- `meters_per_degree_lat`: 111319.9 meters per degree latitude (constant approximation).
- `meters_per_degree_lon`: Calculated as `111319.9 * cos(latitude_in_radians)` to account for longitude distortion due to Earth's curvature.

#### Purpose:
Converts pixel dimensions to real-world meters using metadata from the image file.

---

### **2. Conversion of Pixels to Meters**
This converts pixel measurements to real-world distances.

#### Formula:
- `meters_x = pixels_x * gsd_x`
- `meters_y = pixels_y * gsd_y`

#### Purpose:
Used to determine the width and length of road segments in meters based on their pixel dimensions.

---

### **3. Area Conversion (Pixels to Meters)**
This converts the area in pixels to square meters.

#### Formula:
- `area_meters = area_pixels * gsd_x * gsd_y`

#### Purpose:
Used to calculate the total road area in real-world square meters.

---

### **4. Lane Estimation**
Estimates the number of lanes in a road segment based on its average width.

#### Formula:
- `estimated_lanes = round(mean_width_meters / 3.6)`

#### Purpose:
Assumes a standard lane width of 3.6 meters to approximate the number of lanes.

---

### **5. Statistics Calculation**
For width, length, and area metrics, the following are computed:

#### Mean and Standard Deviation:
- `mean_width_meters = np.mean(widths_meters)`
- `std_width_meters = np.std(widths_meters)`

#### Min and Max:
- `min_width_meters = np.min(widths_meters)`
- `max_width_meters = np.max(widths_meters)`

#### Totals:
- `total_length_meters = sum(lengths_meters)`
- `total_area_meters = sum(areas_meters)`

#### Purpose:
Provides aggregate statistics for the analyzed road segments.

---

### **6. Visualization**
Annotations and statistics are overlaid on the image for better interpretation:
- `cv2.polylines`: Draws road segments.
- `cv2.putText`: Adds text annotations (e.g., road width in meters).


In [ ]:
class RoadAnalyzer:
    def __init__(self, image_path, json_path, main_image_path=None):
        self.image_path = image_path
        self.json_path = json_path
        self.main_image_path = main_image_path  # Path to the main image for GSD extraction
        self.gsd_x, self.gsd_y = self.extract_gsd_from_main_image()

    def extract_gsd_from_main_image(self):
        if self.main_image_path is None:
            raise ValueError("Main image path is not provided for GSD extraction.")
        
        with Image.open(self.main_image_path) as img:
            metadata = img.tag_v2
            
            # Extract ModelPixelScaleTag
            model_pixel_scale = metadata.get(33550)  # 33550 is the tag ID for ModelPixelScaleTag
            
            if model_pixel_scale is None:
                raise ValueError("ModelPixelScaleTag not found in the main image metadata.")
            
            # Convert pixel scale (degrees/pixel) to meters/pixel
            pixel_scale_x, pixel_scale_y = model_pixel_scale[0], model_pixel_scale[1]
            latitude = metadata.get(33922)[4]  # Extract latitude from ModelTiepointTag
            
            # Conversion from degrees to meters
            meters_per_degree_lat = 111319.9  # Approx. meters per degree latitude
            meters_per_degree_lon = meters_per_degree_lat * math.cos(math.radians(latitude))
            
            gsd_x = pixel_scale_x * meters_per_degree_lon  # Meters per pixel in X direction
            gsd_y = pixel_scale_y * meters_per_degree_lat  # Meters per pixel in Y direction
            
            return gsd_x, gsd_y

    def pixels_to_meters(self, pixels_x, pixels_y):
        meters_x = pixels_x * self.gsd_x
        meters_y = pixels_y * self.gsd_y
        return meters_x, meters_y

    def area_to_meters(self, area_pixels):
        return area_pixels * self.gsd_x * self.gsd_y

    def load_data(self):
        self.image = cv2.imread(self.image_path)
        if self.image is None:
            raise ValueError("Could not read image")

        with open(self.json_path, 'r') as f:
            self.coco_data = json.load(f)

        image_filename = self.image_path.split('/')[-1]
        self.image_info = None
        for img in self.coco_data['images']:
            if img['file_name'] == image_filename:
                self.image_info = img
                break

        if self.image_info is None:
            raise ValueError("Image not found in annotations")

        self.annotations = [
            ann for ann in self.coco_data['annotations']
            if ann['image_id'] == self.image_info['id']
        ]

        return len(self.annotations)

    def analyze_roads(self):
        road_segments = []

        for ann in self.annotations:
            x, y, w, h = [int(v) for v in ann['bbox']]

            width_pixels = min(w, h)
            length_pixels = max(w, h)

            width_meters, length_meters = self.pixels_to_meters(width_pixels, length_pixels)

            area_meters = self.area_to_meters(ann['area'])

            if ann['segmentation']:
                polygon = np.array(ann['segmentation'][0]).reshape(-1, 2)

                segment = {
                    'bbox': (x, y, w, h),
                    'polygon': polygon,
                    'area_pixels': ann['area'],
                    'width_pixels': width_pixels,
                    'length_pixels': length_pixels,
                    'width_meters': width_meters,
                    'length_meters': length_meters,
                    'area_meters': area_meters
                }
                road_segments.append(segment)

        return road_segments

    def calculate_statistics(self, road_segments):
        widths_pixels = [seg['width_pixels'] for seg in road_segments]
        widths_meters = [seg['width_meters'] for seg in road_segments]
        lengths_meters = [seg['length_meters'] for seg in road_segments]
        areas_meters = [seg['area_meters'] for seg in road_segments]

        mean_width_meters = np.mean(widths_meters)
        std_width_meters = np.std(widths_meters)
        min_width_meters = np.min(widths_meters)
        max_width_meters = np.max(widths_meters)
        total_length_meters = sum(lengths_meters)
        total_area_meters = sum(areas_meters)

        estimated_lanes = int(np.round(mean_width_meters / 3.6)) # 3.6 meters per lane

        stats = {
            'mean_width_pixels': np.mean(widths_pixels),
            'std_width_pixels': np.std(widths_pixels),
            'mean_width_meters': mean_width_meters,
            'std_width_meters': std_width_meters,
            'min_width_meters': min_width_meters,
            'max_width_meters': max_width_meters,
            'total_length_meters': total_length_meters,
            'total_area_meters': total_area_meters,
            'num_segments': len(road_segments),
            'estimated_lanes': estimated_lanes
        }

        return stats

    def visualize_analysis(self, road_segments, stats):
        result = self.image.copy()

        for segment in road_segments:
            pts = segment['polygon'].astype(np.int32)
            cv2.polylines(result, [pts], True, (0, 255, 0), 2)

            x, y, w, h = segment['bbox']
            center_x = x + w // 2
            center_y = y + h // 2
            cv2.putText(result, f"{segment['width_meters']:.1f}m",
                        (center_x, center_y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

        y_pos = 30
        stats_text = [
            f"Mean Width: {stats['mean_width_meters']:.1f}m",
            f"Estimated Lanes: {stats['estimated_lanes']}",
            f"Total Length: {stats['total_length_meters']:.1f}m",
            f"Total Area: {stats['total_area_meters']:.1f}m²"
        ]

        for text in stats_text:
            cv2.putText(result, text, (10, y_pos),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            y_pos += 25

        return result

    def process_image(self):
        num_annotations = self.load_data()
        print(f"Found {num_annotations} road annotations")

        road_segments = self.analyze_roads()
        stats = self.calculate_statistics(road_segments)
        result = self.visualize_analysis(road_segments, stats)

        # Display the result in the notebook
        result_rgb = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(12, 12))
        plt.imshow(result_rgb)
        plt.axis("off")
        

        return stats


def main():
    image_path = "/home/user/Documents/LRS/dataset/json/tile_17_jpg.rf.7689470d77be236ea213a43f80424eff.jpg"
    json_path = "/home/user/Documents/LRS/dataset/json/_annotations.coco.json"
    main_image_path = "data/20241014aC0822230w275445n.tif"

    analyzer = RoadAnalyzer(image_path, json_path, main_image_path=main_image_path)

    try:
        stats = analyzer.process_image()

        print("\nRoad Analysis Results:")
        print(f"Number of road segments: {stats['num_segments']}")
        print(f"Mean road width: {stats['mean_width_meters']:.1f} meters")
        print(f"Minimum width: {stats['min_width_meters']:.1f} meters")
        print(f"Maximum width: {stats['max_width_meters']:.1f} meters")
        print(f"Estimated number of lanes: {stats['estimated_lanes']} (3.6 meters per lane)")
        print(f"Total road length: {stats['total_length_meters']:.1f} meters")
        print(f"Total road area: {stats['total_area_meters']:.1f} square meters")
        plt.show()
    except Exception as e:
        print(f"Error processing image: {str(e)}")


if __name__ == "__main__":
    main()
